In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pathlib
import pandas as pd
import cv2

In [6]:
def create_depth_im(num_file):
    path = f"trans/{num_file:08}.ply"
    
    arr = np.loadtxt(path)
    arr = arr[:, :3] / 100

    x = arr[:, 0].reshape(-1, 1)
    y = arr[:, 2].reshape(-1, 1)
    z = -arr[:, 1].reshape(-1, 1)

    cp = np.concatenate((x, y, z), axis=1)
    df = pd.DataFrame(data = np.round(cp, 2), columns=["x", "y", "z"])
    data = df.loc[df.groupby(["x", "y"])["z"].idxmax()].to_numpy()

    img = np.zeros((1000, 1000))

    min_x = np.min(data[:, 0])
    min_y = np.min(data[:, 1])

    data[:, 0] = np.clip(200 * (data[:, 0] - min_x + 1), None, 999)
    data[:, 1] = np.clip(200 * (data[:, 1] - min_y + 1), None, 999)
    data[:, 2] = np.clip(data[:, 2] * 100, 0, 255)
    data = data.astype(int)

    x = data[:, 0]
    y = data[:, 1]
    col = data[:, 2]

    for i in range(len(x)):
        img[x[i], y[i]] = col[i]

    cv2.imwrite("tmp.png", img)
    img = cv2.imread("tmp.png", 0)

    tmp = img.copy()
    kernel = np.ones((3, 3),np.uint8)
    tmp = cv2.dilate(tmp,kernel,iterations = 1)
    tmp = cv2.erode(tmp,kernel,iterations = 1)
    im2,contours,hierarchy = cv2.findContours(tmp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if len(contours) != 0:
        # the contours are drawn here
        # cv2.drawContours(output, contours, -1, 100, 2)

        #find the biggest area of the contour
        c = max(contours, key = cv2.contourArea)

        x,y,w,h = cv2.boundingRect(c)

        img[:y, :] = 255
        img[y+h:, :] = 255
        img[:, :x] = 255
        img[:, x+w:] = 255

        kernel = np.ones((3, 3),np.uint8)
        img = cv2.dilate(img, kernel)
        img[img == 0] = 255

        x = x + w//2 - 200
        y = y + h//2 - 200
        w = h = 400
        
        res = img[y: y+h, x: x+w]

        negative_res = 255 - res
        res[res < 255] = negative_res[res < 255]
        
        res = cv2.GaussianBlur(res, (3, 3), 0)

        if res.shape[0] != 400 or res.shape[1] != 400:
            print("problem: ", num_file)
        
        cv2.imwrite(f"depth/{num_file:08}.png", res)

In [7]:
for i in range(10, 29401, 10):
    create_depth_im(i)

KeyboardInterrupt: 